[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mckinsey/vizro/blob/docs/add_colab_link/vizro-ai/examples/chart_by_vizro_ai.ipynb)

# Use Vizro-AI to generate Vizro charts

In [ ]:
# @title Setup
#@markdown <-- click the `Run cell` button to
#@markdown #### 1. Install Vizro-AI
print("Installing vizro-ai")
!pip install vizro-ai -q -q -q
print("✅ vizro-ai installed")

#@markdown #### 2. Secure your LLM API key as environment variable
#@markdown - click the `Secrets` button on the left
#@markdown - click `Add new secret`
#@markdown - insert "OPENAI_API_KEY" under `Name`
#@markdown - insert the API key value under `Value`
#@markdown - [optional] you might also need to insert "OPENAI_BASE_URL" and its values, based on the requirements from your API key provider.
#@markdown - click the `Notebook access` toggle to make it available for the notebook run

import os
from google.colab import userdata

def safe_get_userdata(key):
    try:
        return userdata.get(key)
    except Exception as e:
        print(f"Warning: Unable to access {key}. Reason: {str(e)}")
        return None

# Always try to set the OPENAI_API_KEY
api_key = safe_get_userdata("OPENAI_API_KEY")
if api_key:
    os.environ["OPENAI_API_KEY"] = api_key
else:
    print("❌ OPENAI_API_KEY not set. Click `Secrets` icon on the left to setup.")

# Conditionally set OPENAI_BASE_URL if provided and accessible
openai_base_url = safe_get_userdata("OPENAI_BASE_URL")
if openai_base_url:
    os.environ["OPENAI_BASE_URL"] = openai_base_url
else:
    print("User defined OPENAI_BASE_URL not set. Using default URL.")

print("\nCurrent environment variables:")
print(f"OPENAI_API_KEY: {'✅ Set' if 'OPENAI_API_KEY' in os.environ else 'Not set'}")
print(f"OPENAI_BASE_URL: {'✅ Set' if 'OPENAI_BASE_URL' in os.environ else 'Not set by user, use default'}")

In [ ]:
# @title ## Set user input

#@markdown <-- click the `Run cell` button to
#@markdown #### 1. Choose the LLM model

LLM = 'gpt-3.5-turbo' # @param ["gpt-4o", "gpt-3.5-turbo", "gpt-4-turbo"]

# print(f"Selected LLM: {LLM}")

#@markdown ---
#@markdown #### 2. Upload your CSV data (only 1 dataframe accepted per chart)
#@markdown - click `Choose Files` if you'd like to upload and use your own data
#@markdown - or click `Cancel upload` and use the default example data for initial exploration

import pandas as pd
from plotly.express.data import gapminder
from google.colab import files

uploaded = files.upload()

if not uploaded:
    print("No files uploaded. Use the plotly gapminder dataset as an example.")
    df = gapminder()
else:
    for fn in uploaded.keys():
        print('User uploaded file "{name}"'.format(
            name=fn, length=len(uploaded[fn])))

        df = pd.read_csv(fn)
print(f"Dataframe used for plotting: \n{df.head()}")

#@markdown ---
#@markdown #### 3. Input your chart question

# @markdown #### Instructions:
user_input = 'describe the composition of gdp in continent' # @param {type:"string", placeholder:"Enter the requirements for the chart"}

# print(user_input)

In [ ]:
# @title Create the chart using Vizro-AI

#@markdown <-- click the `Run cell` button to

import ipywidgets as widgets
from IPython.display import display

def run_code(button):
    from vizro_ai import VizroAI

    vizro_ai = VizroAI()
    button1.result = vizro_ai.plot(df, user_input, explain=True)

button1 = widgets.Button(description="Create chart!")
display(button1)

button1.on_click(run_code)

In [ ]:
# @title Show the chart
#@markdown <-- click the `Run cell` button to
button1.result.show()